In [1]:
using DataStructures
using CPUTime
using DelimitedFiles
using Base

In [2]:
numVertex=4
Edges = [(4,1,2), (1,2,1), (2,3,1), (3,4,1), (4,2,3), (1,3,5)]

6-element Array{Tuple{Int64,Int64,Int64},1}:
 (4, 1, 2)
 (1, 2, 1)
 (2, 3, 1)
 (3, 4, 1)
 (4, 2, 3)
 (1, 3, 5)

In [3]:
numVertex=6
Edges = [
(6, 1, 1),
(6, 2, 3),
(1, 2, 1),
(1, 3, 7),
(2, 4, 1),
(1, 4, 3),
(3, 4, 1),
(3, 5, 1),
(4, 5, 6)]
# source, destionation, weight

9-element Array{Tuple{Int64,Int64,Int64},1}:
 (6, 1, 1)
 (6, 2, 3)
 (1, 2, 1)
 (1, 3, 7)
 (2, 4, 1)
 (1, 4, 3)
 (3, 4, 1)
 (3, 5, 1)
 (4, 5, 6)

In [56]:
Edges = Vector{}()
function readDataFromFile(filename="Downloads/in9.txt")
    open(filename, "r") do f 
        n, m = split(readline(f))
        n = parse(Int, n)
        sizehint!(Edges, parse(Int, m))
#         println(n," ",m)
        for ln in eachline(f)
#             a,b,c = split(ln)
            a,b,c = readdlm(IOBuffer(ln), Int)
            if(a==0)
                a=n
            end
            if(b==0)
                b=n
            end
            push!(Edges, (a,b,c))
#             println(a," ",b," ",c)
        end
        return n
    end
end
numVertex = readDataFromFile()

3665

In [74]:
uf = IntDisjointSets(numVertex+1)

weights = Vector{}()
sizehint!(weights, length(Edges))

mst= Vector{}()
sizehint!(mst, numVertex)

0-element Array{Any,1}

In [75]:
for e in Edges
    push!(weights, (e[3], (src=e[1],dst= e[2])))
end

In [76]:
Base.@pure function kruskal(EdgeWeights, MST, UF)
    if isempty(EdgeWeights)
        return 0
    end
    sortedEdgeWeights = sort(EdgeWeights)
    mst_weight=0
    for weight in sortedEdgeWeights
        c, e = weight
        if !in_same_set(uf, e.src, e.dst)
            union!(uf, e.src, e.dst)
            push!(mst, e)    
            mst_weight+=c
        end
    end
    return mst_weight
end    

kruskal (generic function with 1 method)

In [69]:
using Random
rng = MersenneTwister();

In [70]:
Base.@pure function filterKruskal(EdgeWeights, MST, UF)
    if isempty(EdgeWeights)
        return 0
    end
    if kruskalThreshold(length(weights), length(MST))
        tmp =  Threads.@spawn kruskal(EdgeWeights, MST, UF)
        return fetch(tmp)
    else
        sampler = Random.Sampler(rng, EdgeWeights)
        pivot = rand(rng, sampler)
        
        # Partition
        EdgeWeightsLeft = Vector{}()
        EdgeWeightsRight = Vector{}()
        for EdgeWeight in EdgeWeights
            if EdgeWeight[1] < pivot[1]
                push!(EdgeWeightsLeft, EdgeWeight)
            else
                push!(EdgeWeightsRight, EdgeWeight)
            end
        end
#         println("pivot = ",pivot)
#         println("left = ",length(EdgeWeightsLeft))
#         println("right = ",length(EdgeWeightsRight))
        leftMST = Threads.@spawn filterKruskal(EdgeWeightsLeft, MST, UF)
        rightMST = Threads.@spawn filterKruskal(filter(EdgeWeightsRight, UF), MST, UF)
        println(Threads.nthreads())
        return fetch(leftMST) + fetch(rightMST)
    end
end

filterKruskal (generic function with 1 method)

In [71]:
Base.@pure function kruskalThreshold(EdgesLength, MSTLength)
    sp = Random.Sampler(rng, 1:EdgesLength) # or Random.Sampler(MersenneTwister, 1:20)
    return EdgesLength/1.5<=rand(rng, sp) || EdgesLength <=100
end

kruskalThreshold (generic function with 1 method)

In [72]:
Base.@pure function filter(EdgeWeights, UF)
    FilteredEdgeWeights = Vector{}()
    for EdgeWeight in EdgeWeights
        if !in_same_set(UF, EdgeWeight[2].src, EdgeWeight[2].dst)
            push!(FilteredEdgeWeights, EdgeWeight)
        end
    end
    return FilteredEdgeWeights
end

filter (generic function with 1 method)

In [73]:
@time @CPUtime filterKruskal(weights, mst, uf)

elapsed CPU time: 0.132587 seconds
  0.127359 seconds (772.63 k allocations: 21.177 MiB)


1641614

In [38]:
@time @CPUtime kruskal(weights, mst, uf)


elapsed CPU time: 0.087546 seconds
  0.102266 seconds (701.98 k allocations: 17.579 MiB)


1641614

In [14]:
Threads.nthreads()

4